In [3]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [4]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
dekker_mc = [
'51d9ac15-e55e-4f64-85e5-01998e349cf1',
'998573fb-384e-4792-a621-254d705d4ec9',
]

my_exp = dekker_mc[0]

print my_exp
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




51d9ac15-e55e-4f64-85e5-01998e349cf1
file_reference 4a6d10ee-2edb-4402-a98f-0edb1d58f5e9 released to project
workflow 2324ad76-ff37-4157-8bcc-3ce72b7dace9 released to project
workflow d3f25cd3-e726-4b3c-a022-48f844474b41 released to project
file_reference 4a6d10ee-2edb-4402-a98f-0edb1d582084 released to project
file_processed 76037ec6-a227-4e83-84d5-8d7768255cac to be uploaded by workflow
file_processed 9afe35f0-4059-4cee-afc1-c5ea261f3b7b to be uploaded by workflow
file_processed c0e2a6aa-7981-479a-8f45-6894ea5dc307 to be uploaded by workflow
file_processed 45f1bca5-affc-412f-8ab4-3358d850b881 to be uploaded by workflow
262
262


In [34]:
## This part is only for making non file objects "in submission"
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        for raw_data in store[a_type]:
            print '    SKIPPING FILES', a_type, raw_data['status'], raw_data['uuid']
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['submission in progress', 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released"}
            #ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate in review by lab 9b1063f5-c891-44a8-b44a-3b4879257366
quality_metric_fastqc in review by lab 94d04b16-66dc-4470-84af-e37f9c8a8e47
quality_metric_fastqc in review by lab 984a689c-c839-4827-926b-bc2b19b3b327
quality_metric_fastqc in review by lab 4e9da48b-766a-415b-9462-53895a2a5e74
quality_metric_fastqc in review by lab d991f409-37ad-4cd0-a677-27812de0fd5b
quality_metric_fastqc in review by lab d83ebdce-4467-4f2e-a16e-12a7a9babc6d
quality_metric_fastqc in review by lab 1e8b42b4-4a5f-4859-aa75-042720659a75
quality_metric_fastqc in review by lab 94b5c103-3e01-4f43-90ec-b17b312e909f
quality_metric_fastqc in review by lab 9d2ae017-022b-4dc9-9a05-431ecfbf0a7a
quality_metric_fastqc in review by lab 341e7d0d-a837-4b83-a7c2-2f89edfce7ce
quality_metric_fastqc in review by lab ae18f2bd-a6a4-4542-9730-ac3674d2cc33
quality_metric_fastqc in review by lab aadb6589-1087-40ea-a671-742d2b76fcec
quality_metric_fastqc in review by lab b87e2718-b55a-4a2a-95e5-3ce70a383588
quality_m

In [35]:
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            continue        
        if raw_data['status'] not in ['submission in progress', 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "submission in progress"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate in review by lab 9b1063f5-c891-44a8-b44a-3b4879257366
quality_metric_fastqc in review by lab 94d04b16-66dc-4470-84af-e37f9c8a8e47
quality_metric_fastqc in review by lab 984a689c-c839-4827-926b-bc2b19b3b327
quality_metric_fastqc in review by lab 4e9da48b-766a-415b-9462-53895a2a5e74
quality_metric_fastqc in review by lab d991f409-37ad-4cd0-a677-27812de0fd5b
quality_metric_fastqc in review by lab d83ebdce-4467-4f2e-a16e-12a7a9babc6d
quality_metric_fastqc in review by lab 1e8b42b4-4a5f-4859-aa75-042720659a75
quality_metric_fastqc in review by lab 94b5c103-3e01-4f43-90ec-b17b312e909f
quality_metric_fastqc in review by lab 9d2ae017-022b-4dc9-9a05-431ecfbf0a7a
quality_metric_fastqc in review by lab 341e7d0d-a837-4b83-a7c2-2f89edfce7ce
quality_metric_fastqc in review by lab ae18f2bd-a6a4-4542-9730-ac3674d2cc33
quality_metric_fastqc in review by lab aadb6589-1087-40ea-a671-742d2b76fcec
quality_metric_fastqc in review by lab b87e2718-b55a-4a2a-95e5-3ce70a383588
quality_m